In [42]:
import argparse, math, os, random, shutil, sys, textwrap
from argparse import RawTextHelpFormatter
from PIL import Image, ExifTags

In [39]:
d_size = 0.2
d_dist = 20

d_input = "input"
d_output = "output"
d_wm = "logo.png"
prefix = "wm_"
pos_choices = ["bottom-right", "bottom-left", "top-right", "top-left", "center", "random", "all"]

ap = argparse.ArgumentParser(description="ESN Watermark Inserter", formatter_class=RawTextHelpFormatter)
ap.add_argument("-q", 	"--quiet", 		action="store_true",	 						help="silent running")
ap.add_argument("-np", 	"--no-prefix", 	action="store_true", 							help=f"do not add a '{ prefix }' prefix to outputs")
ap.add_argument("-i", 	"--input", 		action="store", type=str,	default=d_input,	help=f"set a custom input directory path (default is '{ d_input }')")
ap.add_argument("-o", 	"--output", 	action="store", type=str,	default=d_output,	help=f"set a custom output directory path (default is '{ d_output }')")
ap.add_argument("-wm", 	"--watermark", 	action="store", type=str,	default=d_wm,		help=f"set a custom watermark file (default is '{ d_wm }')")
ap.add_argument("-r", 	"--ratio", 		action="store", type=float, default=d_size,		help=f"set the ratio of the watermark's size compared to the image's size (default is { d_size })")
ap.add_argument("-d", 	"--distance",	action="store", type=int,	default=d_dist,		help=f"set the distance between the watermark and the edge in pixels (default is { d_dist })")
ap.add_argument("-p", 	"--pos", 		type=str,
                                        default="bottom-right",
                                        metavar="POSITION",
                                        choices=pos_choices,
                                        help=textwrap.dedent("set the position of the watermark, options are the following:\n"
                                                            + "> bottom-right [Default value]\n"
                                                            + "> bottom-left\n"
                                                            + "> top-right\n"
                                                            + "> top-left\n"
                                                            + "> center\n"
                                                            + "> random [Random position in the 5 above for each image]\n"
                                                            + "> all [Generates all 5 versions of each image with suffixes]"))

_StoreAction(option_strings=['-p', '--pos'], dest='pos', nargs=None, const=None, default='bottom-right', type=<class 'str'>, choices=['bottom-right', 'bottom-left', 'top-right', 'top-left', 'center', 'random', 'all'], help='set the position of the watermark, options are the following:\n> bottom-right [Default value]\n> bottom-left\n> top-right\n> top-left\n> center\n> random [Random position in the 5 above for each image]\n> all [Generates all 5 versions of each image with suffixes]', metavar='POSITION')

In [111]:
args = vars(ap.parse_args())

quiet = args["quiet"]
prefix = "" if args["no-prefix"] else prefix
path_in = args["input"]
path_out = args["output"]
path_inv = "invalid"
path_wm = args["watermark"]
ratio = args["ratio"]
dist = args["distance"]
pos = args["pos"]

usage: ipykernel_launcher.py [-h] [-q] [-np] [-i INPUT] [-o OUTPUT] [-wm WATERMARK] [-r RATIO] [-d DISTANCE]
                             [-p POSITION]
ipykernel_launcher.py: error: unrecognized arguments: -f C:\Users\david\AppData\Roaming\jupyter\runtime\kernel-42f50f4e-3e50-4c13-a3c5-f0789d15b0c2.json


SystemExit: 2

In [113]:
path_in = d_input
path_out = d_output
path_wm = d_wm
pos = "top_left"

In [121]:
def export_image(img, wm, pos, dist, file_out):
    short = "".join([x[0] for x in pos.split("-")])
    iW, iH = img.size
    wW, wH = wm.size

    if short == "c":
        spot = ((iW - wW) / 2, (iH - wH) / 2)
    elif short == 'tl':
        spot = (dist, dist)
    elif short == 'tr':
        spot = (iW - wW - dist, dist)
    elif short == 'bl':
        spot = (dist, iH - wH - dist)
    elif short == 'br':
        spot = (iW - wW - dist, iH - wH - dist)
        
    fst, snd = file_out
    img.paste(wm, spot, wm)
    img.save(fst + "_" + short + "." + snd)

In [117]:
fns = os.listdir(input_dir)
this_pos = pos
count, total, invalid = 0, len(fns), 0

exts = ('.jpg', '.png', '.jpeg')
proc_str = "Processing image {} of {} (position: {}, invalid: {})"
end_str = "Processed {} images successfully!"
inv_str = " ({} image(s) failed and moved to 'invalid')"

# Load watermark
wm = Image.open(path_wm)
wW, wH = wm.size

for fn in fns:
    count += 1
    print(proc_str.format(count, total, invalid), end="\r")
    
    # Generate paths
    file_in = os.path.join(input_dir, fn)
    file_out = os.path.join(output_dir, prefix + fn)
    
    # Move picture to 'invalid' if it doesn't have the right file format
    if not any([fn.lower().endswith(ext) for ext in exts]):
        invalid += 1
        shutil.move(file_in, os.path.join(path_inv, fn))
        continue
    
    file_out = file_out.rsplit(".", 1)
        
    # Load image
    img = Image.open(file_in)
    iW, iH = img.size
    wm_ratio = max(wW / iW, wH / iH)

    # Resize watermark only if it doesn't exceed its own resolution
    if wm_ratio > ratio:
        this_w = int(wW * sqrt(ratio / wm_ratio))
        this_h = int(wH * sqrt(ratio / wm_ratio))
        this_wm = wm.resize((this_w, this_h), Image.ANTIALIAS)
        
    # Randomize position if asked
    if pos == "random":
        this_pos = pos_choices[random.randint(0, 4)]

    # Try to re-orient the picture if orientation data is available
    if image._getexif():
        tilt = dict((ExifTags.TAGS[k], v) for k, v in image._getexif().items() if k in ExifTags.TAGS)['Orientation']
        
        try:  
            if tilt == 3:
                img = img.rotate(180, expand=True)
            elif tilt == 6:
                img = img.rotate(270, expand=True)
            elif tilt == 8:
                img = img.rotate(90, expand=True)
        except:
            pass
        
    if pos == "all":
        for c in pos_choices[:5]:
            export_image(img.copy(), wm, c, dist, file_out)
    else:
        export_image(img.copy(), wm, pos, dist, file_out)
    
print(end_str.format(total) + (inv_str.format(invalid) if invalid else ""))

Processed 0 images successfully!


In [33]:
import os
os.listdir("blabla")

FileNotFoundError: [WinError 3] The system cannot find the path specified: 'blabla'